In [7]:
# This code recommends the product url based on image similarity between image generated by GenAI and the corresponding search results on flipkart.It also keeps track of the brands used in user's purchase history.
import pandas as pd
import requests
import os
import matplotlib.pyplot as plt
import io
from PIL import Image
from pathlib import Path
# hashlib allows you to get hashes.
import hashlib
import requests


from bs4 import BeautifulSoup


import csv

In [9]:
#below inputs are taken as per user's purchase history
search_query = "black tshirt for men with flower paint"
brand1 = "NICKY%2BBOY"
brand2 = "Lingo"
brand3 = "EyeBogler"
# product_url1 = "https://www.flipkart.com/eyebogler-striped-men-polo-neck-blue-t-shirt/p/itm186f6204480bb?pid=TSHGB2RYFG7X876S&lid=LSTTSHGB2RYFG7X876SQH5JFB&marketplace=FLIPKART&q=blue+tshirt&store=clo%2Fash%2Fank%2Fedy&spotlightTagId=BestsellerId_clo%2Fash%2Fank%2Fedy&srno=s_1_9&otracker=search&otracker1=search&fm=Search&iid=24209e8e-6846-4613-a9c4-93c66f481c84.TSHGB2RYFG7X876S.SEARCH&ppt=sp&ppn=sp&ssid=51mtef0d3k0000001692522681313&qH=7c91d7904969d325"
# url = "https://www.flipkart.com/search?q="+search_query+"&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
# url = "https://www.flipkart.com/search?q="+search_query+"&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.brand%255B%255D%3D"+brand1+"%2B"+brand2+"&p%5B%5D=facets.brand%255B%255D%3D"+brand3
url = "https://www.flipkart.com/search?q="+search_query+"&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.brand%255B%255D%3DNB%2B"+brand1
r = requests.get(url)
soup = BeautifulSoup(r.text,"html.parser")
# creating a dictionary by mapping product url with image url

image_urls = soup.findAll("img", class_="_2r_T1I")
product_urls = soup.findAll("a", class_="_2UzuFa")

url_mapping = {}

for i in range(30):
    image_url = image_urls[i].get("src")
    product_path = product_urls[i].get("href")
    product_url = "https://www.flipkart.com" + product_path
    url_mapping[image_url] = product_url


In [ ]:
image_url

'https://rukminim2.flixcart.com/image/612/612/xif0q/shirt/e/h/8/2xl-collor-2-aidan-paul-original-imagc3gunm3daexn-bb.jpeg?q=70'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
import requests
import cv2
import numpy as np

# Assuming 'url_mapping' is the dictionary you created earlier
# url_mapping = {image_url: product_url, ...}

# Your test image URL
test_image_path = "/content/generated image.png"
test_image = cv2.imread(test_image_path)
gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
histogram = cv2.calcHist([gray_image], [0],
                         None, [1024], [0, 1024])

# Calculate Euclidean distance between two histograms
def euclidean_distance(hist1, hist2):
    return np.sqrt(np.sum((hist1 - hist2) ** 2))

# List to store top N similar products
num_top_similar = 5
top_similar_products = []

for image_url, product_url in url_mapping.items():
    response = requests.get(image_url)
    image_content = response.content
    nparr = np.frombuffer(image_content, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    gray_image1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    histogram1 = cv2.calcHist([gray_image1], [0], None, [1024], [0, 1024])

    # Calculate Euclidean distance (similarity)
    similarity = euclidean_distance(histogram1, histogram)

    # Add to the list of top similar products
    if len(top_similar_products) < num_top_similar:
        top_similar_products.append((product_url, similarity))
        top_similar_products.sort(key=lambda x: x[1], reverse=True)
    else:
        if similarity < top_similar_products[0][1]:
            top_similar_products[0] = (product_url, similarity)
#             top_similar_products.sort(key=lambda x: x[1], reverse=True)
            top_similar_products.sort(key=lambda x: x[1])


# Extract the product URLs from the list of top similar products
best_product_urls = [product_url for product_url, _ in top_similar_products]

# Now best_product_urls contains the URLs of the top N most similar products
print("Top 5 Best Product URLs:")
for i, product_url in enumerate(best_product_urls):
    print(f"{i+1}: {product_url}")


Top 5 Best Product URLs:
1: https://www.flipkart.com/nb-nicky-boy-typography-men-round-neck-black-t-shirt/p/itmaff3b194fc6e3?pid=TSHGFRRYZSNYMFAZ&lid=LSTTSHGFRRYZSNYMFAZEXLNPF&marketplace=FLIPKART&q=black+tshirt+for+men+with+flower+paint&store=clo%2Fash%2Fank%2Fedy&srno=s_1_4&otracker=search&otracker1=search&fm=organic&iid=aa67195c-ed91-4593-a954-9b0e0409de9f.TSHGFRRYZSNYMFAZ.SEARCH&ppt=None&ppn=None&ssid=ftk57qexj40000001692570619193&qH=cc8ea4206929ce42
2: https://www.flipkart.com/nb-nicky-boy-printed-men-round-neck-black-t-shirt/p/itm3a9533d26d24f?pid=TSHGJZTGTGXE27EP&lid=LSTTSHGJZTGTGXE27EP18ERAF&marketplace=FLIPKART&q=black+tshirt+for+men+with+flower+paint&store=clo%2Fash%2Fank%2Fedy&srno=s_1_5&otracker=search&otracker1=search&fm=organic&iid=aa67195c-ed91-4593-a954-9b0e0409de9f.TSHGJZTGTGXE27EP.SEARCH&ppt=None&ppn=None&ssid=ftk57qexj40000001692570619193&qH=cc8ea4206929ce42
3: https://www.flipkart.com/nb-nicky-boy-printed-men-hooded-neck-black-t-shirt/p/itm64736f9c28c36?pid=TSHGM98S